# 导入库函数

In [1]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import Parameter
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
from torch_geometric.datasets import Planetoid
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# evaluating indicator

In [2]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    ap = average_precision_score(true_edges, predict_edges)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    return ap, auc
    
    #predict_edges[predict_edges>gama] = 1
    #predict_edges[predict_edges<=gama] = 0
    
    #print("ACC： ",accuracy_score(true_edges, predict_edges))
    #print("AP： ",precision_score(true_edges, predict_edges, average='macro'))
    #print("RECALL： ",recall_score(true_edges, predict_edges, average='macro'))
    #print("F1 SCORE： ",f1_score(true_edges, predict_edges, average='macro'))

# model

In [3]:
"""  Given the cluster centroids, update the node representation  """
class reconstruction_graph(nn.Module):
    def __init__(self, NE, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, CC):
        # Calculate the attention score (in nodes) for each node to each cluster
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # Distance from the node to the center of mass
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power    
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t()       #soft assignment using t-distribution
        
        # Calculate the cluster similarity between nodes (cosine similarity): the larger the value, the more conducive to edge formation
        prod = torch.mm(soft_assignments, soft_assignments.t())       #  numerator
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)      #    denominator 
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # Calculate the distance between nodes: smaller values favor edge formation
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # The formation probability of the edges is calculated
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=0.4, momentum=0.9)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, CC, edge_train, edge_test):
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        ap, auc = predict_acc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach(), ap, auc
    
"""  Type-centroid update aimed at maximizing the prediction effect   """
class ClusteringLayer(nn.Module):
    """  Given the node representation, update the cluster centroids  """
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, NE):
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2) 
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power   
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() 
        
        prod = torch.mm(soft_assignments, soft_assignments.t())
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(NE, CC, g, edge_train, edge_test, num_epochs=5):        
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=0.4, momentum=0.9)
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    ap, auc = predict_acc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach(), ap, auc

In [5]:
def get_cora_graph(filename_adj, nodes_number):
    raw_edges = pd.read_csv(filename_adj, header=None, sep='\t')
    
    drop_self_loop = raw_edges[raw_edges[0]!=raw_edges[1]]    # 去掉自循环
    
    graph_np = np.zeros((nodes_number,nodes_number))
    for i in range(drop_self_loop.shape[0]):
        graph_np[drop_self_loop.iloc[i,0],drop_self_loop.iloc[i,1]]=1
        graph_np[drop_self_loop.iloc[i,1],drop_self_loop.iloc[i,0]]=1

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # 只取左上角矩阵
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # 只取左上角矩阵
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.9),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.9)*4,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [6]:
filename = "datasets/wiki.txt"
nodes_number = 2405

In [7]:
embedding_dim = 24
n_clusters = 88
alpha = 1
beta = 4.2
DEC = 5.0

all_ap_first_ap = []
all_ap_first_auc = []

all_auc_first_ap = []
all_auc_first_auc = []

for i in range(10):
    
    train_edge, test_edge, train_mask, test_mask = get_cora_graph(filename, nodes_number)
    
    ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)       
    raw_nodes_embedding = ini_embedding.weight.detach()        # Initial node embedding

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(raw_nodes_embedding)
    cluster_centers = kmeans.cluster_centers_
    raw_cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)     # Initial cluster centroids
    
    ap_first__ap = 0
    ap_first_auc = 0
    auc_first__ap = 0
    auc_first_auc = 0
    
    for module_epoch in range(400):
        print("######################### Module cycle ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
        raw_nodes_embedding, nodes_ap, nodes_auc = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
        if nodes_ap > ap_first__ap:
            ap_first__ap = nodes_ap
            ap_first_auc = nodes_auc
        if nodes_auc > auc_first_auc:
            auc_first_auc = nodes_auc
            auc_first__ap = nodes_ap
            
        raw_cluster_centers, cluster_ap, cluster_auc = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)
        if cluster_ap > ap_first__ap:
            ap_first__ap = cluster_ap
            ap_first_auc = cluster_auc
        if cluster_auc > auc_first_auc:
            auc_first_auc = cluster_auc
            auc_first__ap = cluster_ap
    
    all_ap_first_ap.append(ap_first__ap)
    all_ap_first_auc.append(ap_first_auc)
    all_auc_first_ap.append(auc_first__ap)
    all_auc_first_auc.append(auc_first_auc)

######################### Module cycle ： 0 ##########################
AP：  0.6858151025762429
AUC SCORE:  0.6596473692033294
AP：  0.6967201305942616
AUC SCORE:  0.6624357164090369
######################### Module cycle ： 1 ##########################
AP：  0.7382871414336675
AUC SCORE:  0.7151155618311535
AP：  0.7479142623309186
AUC SCORE:  0.7182316438763376
######################### Module cycle ： 2 ##########################
AP：  0.7649874271830183
AUC SCORE:  0.7310909631391203
AP：  0.7704917501247893
AUC SCORE:  0.7372406361474435
######################### Module cycle ： 3 ##########################
AP：  0.7816234565258668
AUC SCORE:  0.748163644470868
AP：  0.7846126108420737
AUC SCORE:  0.7502757134363853
######################### Module cycle ： 4 ##########################
AP：  0.789627962950201
AUC SCORE:  0.7549256837098692
AP：  0.7928180788844927
AUC SCORE:  0.7597484393579073
######################### Module cycle ： 5 ##########################
AP：  0.7963584365349777
AUC SCOR

AP：  0.9093468221540768
AUC SCORE:  0.8863592449464923
######################### Module cycle ： 46 ##########################
AP：  0.9105059174722268
AUC SCORE:  0.8880610879904876
AP：  0.9108065535739948
AUC SCORE:  0.8885619797859692
######################### Module cycle ： 47 ##########################
AP：  0.9113235506188234
AUC SCORE:  0.8891041171224732
AP：  0.911680860245836
AUC SCORE:  0.8896157847800237
######################### Module cycle ： 48 ##########################
AP：  0.9122333578640803
AUC SCORE:  0.8901166765755053
AP：  0.9122473888711031
AUC SCORE:  0.8898647443519621
######################### Module cycle ： 49 ##########################
AP：  0.9132184856825907
AUC SCORE:  0.8913793103448275
AP：  0.9135995795384084
AUC SCORE:  0.891872770511296
######################### Module cycle ： 50 ##########################
AP：  0.9139619236223019
AUC SCORE:  0.8922019916765755
AP：  0.9142475136740451
AUC SCORE:  0.8924650713436386
######################### Module cycle ： 5

AP：  0.9305085916188952
AUC SCORE:  0.9124829072532699
AP：  0.9306269689400517
AUC SCORE:  0.9126664684898931
######################### Module cycle ： 92 ##########################
AP：  0.9309755975761179
AUC SCORE:  0.9131391200951247
AP：  0.9310345669938178
AUC SCORE:  0.9132178953626634
######################### Module cycle ： 93 ##########################
AP：  0.9311344858875046
AUC SCORE:  0.9133130202140309
AP：  0.9310530142983188
AUC SCORE:  0.9131428359096313
######################### Module cycle ： 94 ##########################
AP：  0.9312628274998666
AUC SCORE:  0.9135827883472056
AP：  0.9313536034049342
AUC SCORE:  0.9137656064209275
######################### Module cycle ： 95 ##########################
AP：  0.9315605639974865
AUC SCORE:  0.9139840963139121
AP：  0.9315497678235266
AUC SCORE:  0.9138882282996432
######################### Module cycle ： 96 ##########################
AP：  0.9315631269760255
AUC SCORE:  0.9139967300832342
AP：  0.9312910478758821
AUC SCORE:  0.91

AP：  0.9363593665542957
AUC SCORE:  0.921273781212842
######################### Module cycle ： 137 ##########################
AP：  0.9366194641716656
AUC SCORE:  0.921571046373365
AP：  0.9366996192772042
AUC SCORE:  0.9215502378121285
######################### Module cycle ： 138 ##########################
AP：  0.9365919677125705
AUC SCORE:  0.9214759215219976
AP：  0.9363430110722597
AUC SCORE:  0.9210508323424496
######################### Module cycle ： 139 ##########################
AP：  0.9365767191737921
AUC SCORE:  0.9215621284185493
AP：  0.9365819268528581
AUC SCORE:  0.9216542806183116
######################### Module cycle ： 140 ##########################
AP：  0.9368192508744666
AUC SCORE:  0.9219277645659928
AP：  0.9369370649026424
AUC SCORE:  0.9219389120095125
######################### Module cycle ： 141 ##########################
AP：  0.9368106671662382
AUC SCORE:  0.9218452734839477
AP：  0.9366113211301074
AUC SCORE:  0.9214870689655172
######################### Module cycl

AP：  0.9396053920020232
AUC SCORE:  0.9259995541022592
AP：  0.9396354852495247
AUC SCORE:  0.9259906361474435
######################### Module cycle ： 183 ##########################
AP：  0.9396241851813287
AUC SCORE:  0.9259519916765755
AP：  0.9394855762669144
AUC SCORE:  0.9256725624256836
######################### Module cycle ： 184 ##########################
AP：  0.9395777960165556
AUC SCORE:  0.9259980677764565
AP：  0.9394552814862845
AUC SCORE:  0.9259683412604043
######################### Module cycle ： 185 ##########################
AP：  0.93977206701123
AUC SCORE:  0.9262626337693223
AP：  0.9397972060600355
AUC SCORE:  0.9262225029726516
######################### Module cycle ： 186 ##########################
AP：  0.9398041341129273
AUC SCORE:  0.9262173008323424
AP：  0.9396637056864601
AUC SCORE:  0.9259326694411414
######################### Module cycle ： 187 ##########################
AP：  0.9397317768959681
AUC SCORE:  0.9262068965517242
AP：  0.9396206997696707
AUC SCORE:  0

...

AP：  0.9436707319382702
AUC SCORE:  0.9373260998810942
AP：  0.9437312790097412
AUC SCORE:  0.9374888525564804
######################### Module cycle ： 185 ##########################
AP：  0.9435612013671592
AUC SCORE:  0.9373877824019026
AP：  0.9435922324020936
AUC SCORE:  0.9375334423305588
######################### Module cycle ： 186 ##########################
AP：  0.9434313409186764
AUC SCORE:  0.9374264268727706
AP：  0.9434539567804002
AUC SCORE:  0.9375631688466112
######################### Module cycle ： 187 ##########################
AP：  0.9433307352057243
AUC SCORE:  0.9374791914387635
AP：  0.9432845408147605
AUC SCORE:  0.9375943816884661
######################### Module cycle ： 188 ##########################
AP：  0.9431100683790438
AUC SCORE:  0.9375208085612367
AP：  0.9431255474348896
AUC SCORE:  0.9376508620689655
######################### Module cycle ： 189 ##########################
AP：  0.9429825370061004
AUC SCORE:  0.9375728299643282
AP：  0.9430160814852641
AUC SCORE: 

AP：  0.9429548487828314
AUC SCORE:  0.9394827586206898
######################### Module cycle ： 230 ##########################
AP：  0.9429346936672411
AUC SCORE:  0.9394574910820451
AP：  0.9429586619606148
AUC SCORE:  0.9395065398335315
######################### Module cycle ： 231 ##########################
AP：  0.9429474510420526
AUC SCORE:  0.9394849881093936
AP：  0.9429691795185544
AUC SCORE:  0.9395325505350772
######################### Module cycle ： 232 ##########################
AP：  0.9429513976876683
AUC SCORE:  0.9395139714625447
AP：  0.9430041406147307
AUC SCORE:  0.93958531510107
######################### Module cycle ： 233 ##########################
AP：  0.9429851451081994
AUC SCORE:  0.9395615338882283
AP：  0.9430222593707762
AUC SCORE:  0.9396276753864446
######################### Module cycle ： 234 ##########################
AP：  0.9430126335789553
AUC SCORE:  0.9396061236623069
AP：  0.9430438172286416
AUC SCORE:  0.9396655766944114
######################### Module cycl

AP：  0.9438985346417716
AUC SCORE:  0.9411593341260405
AP：  0.9439326422229314
AUC SCORE:  0.9411987217598098
######################### Module cycle ： 276 ##########################
AP：  0.943906936243122
AUC SCORE:  0.9411831153388823
AP：  0.9439457274655794
AUC SCORE:  0.9412247324613556
######################### Module cycle ： 277 ##########################
AP：  0.943920905404814
AUC SCORE:  0.9412135850178357
AP：  0.9439623475070789
AUC SCORE:  0.9412566884661118
######################### Module cycle ： 278 ##########################
AP：  0.9439398176642332
AUC SCORE:  0.9412373662306778
AP：  0.9440027433157512
AUC SCORE:  0.9413035077288943
######################### Module cycle ： 279 ##########################
AP：  0.9439725325474069
AUC SCORE:  0.9412864149821641
AP：  0.9440097717466704
AUC SCORE:  0.9413284036860879
######################### Module cycle ： 280 ##########################
AP：  0.9439958700453452
AUC SCORE:  0.9413220868014268
AP：  0.9440374723926396
AUC SCORE:  0

AP：  0.9445970356842298
AUC SCORE:  0.9426062722948872
######################### Module cycle ： 321 ##########################
AP：  0.9445895342631574
AUC SCORE:  0.9426025564803806
AP：  0.9446036124314996
AUC SCORE:  0.9426174197384068
######################### Module cycle ： 322 ##########################
AP：  0.9446023711398257
AUC SCORE:  0.9426241082045185
AP：  0.9446190821878222
AUC SCORE:  0.9426419441141498
######################### Module cycle ： 323 ##########################
AP：  0.9446198391208065
AUC SCORE:  0.9426560642092746
AP：  0.9446370424840534
AUC SCORE:  0.9426716706302022
######################### Module cycle ： 324 ##########################
AP：  0.9446349635894263
AUC SCORE:  0.9426850475624257
AP：  0.9446309540011707
AUC SCORE:  0.9426746432818074
######################### Module cycle ： 325 ##########################
AP：  0.9446426293272931
AUC SCORE:  0.942712544589774
AP：  0.9446359193478469
AUC SCORE:  0.9426991676575505
######################### Module cyc

AP：  0.9455365575469511
AUC SCORE:  0.9442404875148633
AP：  0.9455405661334237
AUC SCORE:  0.9442442033293698
######################### Module cycle ： 367 ##########################
AP：  0.9455466394977273
AUC SCORE:  0.944254607609988
AP：  0.9455437892167752
AUC SCORE:  0.9442605529131985
######################### Module cycle ： 368 ##########################
AP：  0.9455407831464684
AUC SCORE:  0.9442717003567181
AP：  0.9455509558090076
AUC SCORE:  0.9442873067776456
######################### Module cycle ： 369 ##########################
AP：  0.9455488304063746
AUC SCORE:  0.9442880499405469
AP：  0.945557306903587
AUC SCORE:  0.9443073721759809
######################### Module cycle ： 370 ##########################
AP：  0.9455705882378405
AUC SCORE:  0.9443289239001189
AP：  0.945568245231901
AUC SCORE:  0.9443304102259216
######################### Module cycle ： 371 ##########################
AP：  0.9455571286314082
AUC SCORE:  0.9443393281807373
AP：  0.9455542169253122
AUC SCORE:  0.

In [8]:
print("################################  SUM UP #####################################")
print("AP FIRST AP MEAN: ", np.array(all_ap_first_ap).mean())
print("AP FIRST AP STD: ", np.array(all_ap_first_ap).std())

print("AP FIRST AUC MEAN: ", np.array(all_ap_first_auc).mean())
print("AP FIRST AUC STD: ", np.array(all_ap_first_auc).std())

print("AUC FIRST AP MEAN: ", np.array(all_auc_first_ap).mean())
print("AUC FIRST AP STD: ", np.array(all_auc_first_ap).std())

print("AUC FIRST AUC MEAN: ", np.array(all_auc_first_auc).mean())
print("AUC FIRST AUC STD: ", np.array(all_auc_first_auc).std())

################################  SUM UP #####################################
AP FIRST AP MEAN:  0.9427096697036819
AP FIRST AP STD:  0.006319980781755404
AP FIRST AUC MEAN:  0.9381712247324613
AP FIRST AUC STD:  0.0077710848295009325
AUC FIRST AP MEAN:  0.9425732192265208
AUC FIRST AP STD:  0.00642163677098243
AUC FIRST AUC MEAN:  0.9387156658739597
AUC FIRST AUC STD:  0.0071870905863786454
